In [ ]:
#V4
import pandas as pd
import json
from pymongo import MongoClient
from kahi_impactu_utils.Utils import doi_processor
from unidecode import unidecode
from joblib import Parallel, delayed

# MongoDB Configuration
MONGO_URI = "mongodb://localhost:27017/"  # Replace with your MongoDB URI
DB_NAME = "kahi_dev"
COLLECTION_NAME = "person"
DB_OPENALEX = "openalex"
OPENALEX_COLLECTION_NAME = "works"

def read_excel_file(file_path):
    return pd.read_excel(file_path)

def read_csv_file(file_path):
    return pd.read_csv(file_path)

def extract_works_dois(filtered_works):
    works_dois = [row['Doi'] for _, row in filtered_works.iterrows() if doi_processor(row['Doi'])]
    return works_dois

def analyze_works(works_dois, mongo_uri, db_name, collection_name):
    with MongoClient(mongo_uri) as client:
        db = client[db_name]
        collection = db[collection_name]
        works_authorships = []

        for doi in works_dois:
            work_db = collection.find_one({"doi": "https://doi.org/"+doi}, {"authorships.author"})
            if work_db:
                works_authorships.append({"doi": doi, "authorships": work_db["authorships"]})

    return works_authorships

def get_author_identifiers(authors_dicts, works_authorships):
    return

def create_authors_dicts(authors_list):
    author_identifiers = []
    for author in authors_list:
        entry = {
            "author_full_name": author,
            "primer_apellido": "",
            "segundo_apellido": "",
            "nombres": "",
            "author_full_name_normalized": unidecode(author.lower()),
            "author_ids": [],
        }
        author_identifiers.append(entry)
    return author_identifiers

# Main Script
def main():
    # File paths
    excel_file_path = "UNAL_13_01_2025.xlsx"
    csv_file_path = "Lista de docentes investigadores.csv"
    
    works_authorships = []

    # Read Excel file
    data = read_excel_file(excel_file_path)

    # Filter rows with valid DOI
    filtered_data = data[data['Doi'].notnull()]

    # Read CSV file
    csv_authors = read_csv_file(csv_file_path)

    authors_dicts = create_authors_dicts(csv_authors['Nombre'].tolist())
    print(f'{len(authors_dicts)} authors have been extracted\n')

    #works_dois = extract_works_dois(data)
    #print(f'{len(works_dois)} DOI have been extracted\n')

    #works_authorships = analyze_works(works_dois, MONGO_URI, DB_OPENALEX, OPENALEX_COLLECTION_NAME)

    #get_author_identifiers(authors_dicts, works_authorships)
    works_authorships = []
    #get_author_identifiers(authors_dicts)

    # Output result
    return authors_dicts, works_authorships

In [ ]:
%%time
authors_dicts, works_authorships = main()

In [ ]:
db = dbclient["parser_unal"]
authors_collection = db["authors"]
works_collection = db["works"]

authors_collection.insert_many(authors_dicts)
#works_collection.insert_many(works_authorships)

authors_collection.count_documents({}), works_collection.count_documents({})

In [ ]:
def read_json_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return json.load(file)

json_file_path = "unal_authors.json"

# Read json file
json_authors = read_json_file(json_file_path)

def process_one_author(author_doc):
    normalized_name_1 = author_doc["author_full_name_normalized"]
    set1 = set(normalized_name_1.split())

    for author in json_authors:
        display_name = author["full_name"]
        normalized_name_2 = unidecode(display_name.lower())
        set2 = set(normalized_name_2.split())

        if not set1.difference(set2) and not set2.difference(set1):
            # Evita duplicados en 'author_ids'
            author_doc["primer_apellido"] = author["last_names"][0],
            author_doc["segundo_apellido"] = author["last_names"][1] if len(author["last_names"]) > 1 else "",
            author_doc["nombres"] = author["first_names"]
            cod_rh = author["external_ids"]["id"]["COD_RH"]
            rh_id = {"source": "scienti", "id": cod_rh}
            if rh_id not in author_doc["author_ids"]:
                author_doc["author_ids"].append(rh_id)
                #print(cod_rh)

    authors_collection.update_one(
    {"_id": author_doc["_id"]},
    {"$set": {
        "primer_apellido": author_doc["primer_apellido"],
        "segundo_apellido": author_doc["segundo_apellido"],
        "nombres": author_doc["nombres"],
        "author_ids": author_doc["author_ids"]
    }}
)

client = MongoClient("mongodb://localhost:27017/")
# All authors from json file
db = client["parser_unal"]
authors_collection = db["authors"]
authors_list = list(authors_collection.find({}))
Parallel(
    n_jobs=72,
    verbose=5,
    backend="multiprocessing",
)(
    delayed(process_one_author)(author) for author in authors_list)

In [ ]:
# All authors from JSON file - Partial Name with one name
from collections import Counter

def extract_remaining_name(full_name, first_lastname, second_lastname, names):
    full_name_parts = full_name.split()
    for lastname in first_lastname:
        if unidecode(lastname) in full_name_parts:
            full_name_parts.remove(unidecode(lastname))
    for lastname in second_lastname:
        if unidecode(lastname) in full_name_parts:
            full_name_parts.remove(unidecode(lastname))     
    for name in names:
        if unidecode(name) in full_name_parts:
            full_name_parts.remove(unidecode(name))
    return full_name_parts if full_name_parts else None

def read_json_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return json.load(file)

json_file_path = "unal_authors.json"

# Read json file
json_authors = read_json_file(json_file_path)

def process_one_author(author_doc):
    normalized_name_1 = author_doc["author_full_name_normalized"]
    normalized_name_1_split = normalized_name_1.split()
    #set1 = set(normalized_name_1_split)

    for author in json_authors:
        display_name = author["full_name"]
        normalized_name_2 = unidecode(display_name.lower())
        normalized_name_2_split = normalized_name_2.split()
        #set2 = set(normalized_name_2_split)

        if not list((Counter(normalized_name_2_split) - Counter(normalized_name_1_split)).elements()):
            # Evita duplicados en 'author_ids'
            author_doc["primer_apellido"] = [author["last_names"][0].strip()]
            author_doc["segundo_apellido"] = [author["last_names"][1].strip()] if len(author["last_names"]) > 1 else []
            author_doc["nombres"] = author["first_names"]

            if author_doc["primer_apellido"] and author_doc["segundo_apellido"] and author_doc["nombres"]:
                remaining_name = extract_remaining_name(author_doc["author_full_name"], author_doc["primer_apellido"], author_doc["segundo_apellido"], author_doc["nombres"])
                if remaining_name:
                    for name in remaining_name:
                        author_doc["nombres"] = list(author_doc["nombres"])
                        author_doc["nombres"].append(name)

            cod_rh = author["external_ids"]["id"]["COD_RH"]
            rh_id = {"source": "scienti", "id": cod_rh}
            if rh_id not in author_doc["author_ids"]:
                author_doc["author_ids"].append(rh_id)
                #print(cod_rh)

    authors_collection.update_one(
    {"_id": author_doc["_id"]},
    {"$set": {
        "primer_apellido": author_doc["primer_apellido"],
        "segundo_apellido": author_doc["segundo_apellido"],
        "nombres": author_doc["nombres"],
        "author_ids": author_doc["author_ids"]
    }}
)

client = MongoClient("mongodb://localhost:27017/")
# All authors from json file
db = client["parser_unal"]
authors_collection = db["authors"]
authors_list = list(authors_collection.find({"author_ids": {"$eq": []}}))
Parallel(
    n_jobs=72,
    verbose=5,
    backend="multiprocessing",
)(
    delayed(process_one_author)(author) for author in authors_list)

In [ ]:
# Authors without ids vs authors with scienti ids
def process_one_author(author_doc):
    normalized_name_1 = unidecode(author_doc["author_full_name"].lower())
    set1 = set(normalized_name_1.split())

    for min_auth in db_authors_minciencias:
        display_name = min_auth["full_name"]
        normalized_name_2 = unidecode(display_name.lower())
        set2 = set(normalized_name_2.split())
    
        if not set1.difference(set2) and not set2.difference(set1):
            # Evita duplicados en 'author_ids'
            author_doc["primer_apellido"] = min_auth["last_names"][0],
            author_doc["segundo_apellido"] = min_auth["last_names"][1] if len(min_auth["last_names"]) > 1 else "",
            author_doc["nombres"] = " ".join(min_auth["first_names"]),
    
            cod_rh = next((exid["id"]["COD_RH"] for exid in min_auth["external_ids"] if exid["source"] == "scienti" or exid["source"] == "minciencias"), "")
            if cod_rh:
                rh_id = {"source": "scienti", "id": cod_rh} 
                if rh_id not in author_doc["author_ids"]:
                    author_doc["author_ids"].append(rh_id)
    
    authors_collection.update_one(
        {"_id": author_doc["_id"]},
        {"$set": {
            "primer_apellido": author_doc["primer_apellido"],
            "segundo_apellido": author_doc["segundo_apellido"],
            "nombres": author_doc["nombres"],
            "author_ids": author_doc["author_ids"]
        }}
    )

client = MongoClient("mongodb://localhost:27017/")
# Authors without ids vs authors with scienti ids
db = client["parser_unal"]
authors_collection = db["authors"]
authors_list = list(authors_collection.find({"author_ids": {"$eq": []}}))
db_authors_minciencias = list(client["kahi_dev_stable"]["person"].find({"external_ids.source": "scienti"}, {"full_name", "first_names", "last_names", "external_ids"}))

Parallel(
    n_jobs=72,
    verbose=5,
    backend="multiprocessing",
)(
    delayed(process_one_author)(author) for author in authors_list)

In [ ]:
# Filtrar solo los trabajos de autores de los cuales se asocio COD_RH
authors_list = list(authors_collection.find({"author_ids": {"$ne": []}}))
valid_authors = [author["author_full_name"] for author in authors_list]
source_file = "/home/fvergara/projects/colav/data/UNAL/UNAL_13_01_2025.xlsx"
df = pd.read_excel(source_file)

df_filtered = df[df["Coautores"].astype(str).apply(lambda x: any(author in x.split(", ") for author in valid_authors))]

output_file = "UNAL_13_01_2025_filtered.xlsx"
df_filtered.to_excel(output_file, index=False)

In [ ]:
# Crear coleccion de mongodb con los registros de los trabajos
from kahi_impactu_utils.Utils import doi_processor
import pandas as pd
import datetime
from pymongo import MongoClient

def read_excel(file_path):
    return pd.read_excel(file_path)

def get_mongo_client(uri="mongodb://localhost:27017/"):
    client = MongoClient(uri)
    return client

def search_author(author, external_authors_collection):
    return external_authors_collection.find_one({"author_full_name": author})

def insert_staff_record(staff_collection, author_data, cod_rh):
    # Se verifica que no exista un autor con el mismo identificador
    if staff_collection.find_one({"identificación": cod_rh}):
        # Ya existe: no se inserta de nuevo
        return

    # Se arma el registro de STAFF. Se utiliza el campo "author_full_name" para almacenar el nombre completo.
    staff_record = {
        # "author_full_name": author_data.get("author_full_name", ""),
        "tipo_documento": "COD_RH",
        "identificación": cod_rh,
        # Se asume que en author_data los apellidos y nombres pueden venir como lista, por lo que se hace join
        "primer_apellido": " ".join(author_data.get("primer_apellido", [])) if isinstance(author_data.get("primer_apellido"), list) else author_data.get("primer_apellido", ""),
        "segundo_apellido": " ".join(author_data.get("segundo_apellido", [])) if isinstance(author_data.get("segundo_apellido"), list) else author_data.get("segundo_apellido", ""),
        "nombres": " ".join(author_data.get("nombres", [])) if isinstance(author_data.get("nombres"), list) else author_data.get("nombres", ""),
        "nivel_académico": "",
        "tipo_contrato": "",
        "jornada_laboral": "",
        "categoría_laboral": "",
        "sexo": "",
        "fecha_nacimiento": "",
        "fecha_inicial_vinculación": "",
        "fecha_final_vinculación": "",
        "código_unidad_académica": "",
        "unidad_académica": "",
        "código_subunidad_académica": "",
        "subunidad_académica": "",
    }
    staff_collection.insert_one(staff_record)

def insert_ciarp_record(ciarp_collection, row, cod_rh):
    # Se obtiene el año a partir del campo Fecha (formato "%d/%m/%Y")
    try:
        date_obj = datetime.datetime.strptime(row["Fecha"], "%d/%m/%Y")
        year_str = str(date_obj.year)
    except Exception:
        year_str = ""

    ciarp_record = {
        "código_unidad_académica": "",
        "código_subunidad_académica": "",
        "tipo_documento": "COD_RH",
        "identificación": cod_rh,
        "año": year_str,
        "título": row["Título original"],
        "idioma": row["Idioma"],
        "revista": row["Revista / Conferencia"],
        "editorial": "",
        "doi": row["Doi"],  # Puedes optar por usar el DOI procesado (variable doi) si lo deseas.
        "issn": row["ISSN"],
        "isbn": row["ISBN"],
        "volumen": "",
        "issue": "",
        "primera_página": "",
        "pais_producto": "",
        "última_página": "",
        "entidad_premiadora": "",
        "ranking": row["Tipo"],
    }
    ciarp_collection.insert_one(ciarp_record)

def process_data_and_insert(dataframe, external_authors_collection, staff_collection, ciarp_collection):
    for _, row in dataframe.iterrows():
        # Procesa el DOI (si es una cadena válida)
        doi = ""
        if isinstance(row["Doi"], str):
            doi = doi_processor(row["Doi"])
            # Actualiza el valor del DOI en la fila si se requiere:
            row["Doi"] = doi

        # Separa los coautores (asumiendo que vienen separados por ", ")
        coauthors = str(row["Coautores"]).split(", ")

        for author in coauthors:
            # Busca los datos completos del autor en la colección externa
            author_data = search_author(author, external_authors_collection)
            if not author_data:
                # Si el autor no existe en la fuente externa, se omite este registro
                continue

            # Se extrae el código RH (identificador) a partir de la lista de ids
            cod_rh = next((exid["id"] for exid in author_data.get("author_ids", []) if "id" in exid), "")
            if not cod_rh:
                continue

            # Inserta en la colección staff si aún no existe
            insert_staff_record(staff_collection, author_data, cod_rh)
            # Inserta el registro de producción en la colección ciarp
            insert_ciarp_record(ciarp_collection, row, cod_rh)

def main(input_file):
    # Leer el archivo Excel
    data = read_excel(input_file)
    print(f"Source Dataframe shape: {data.shape}")

    # Conectar a MongoDB y definir la base de datos y colecciones
    client = get_mongo_client("mongodb://localhost:27017/")
    db = client["parser_unal"]

    # Se asume que en la base de datos existe una colección con los datos completos de autores.
    authors_collection = db["authors"]

    # Estas serán las colecciones donde se insertarán los registros procesados.
    staff_collection = db["staff"]
    ciarp_collection = db["ciarp"]

    # Procesar la información e insertar en las colecciones
    process_data_and_insert(data, authors_collection, staff_collection, ciarp_collection)

    print("Processing complete. Data inserted into MongoDB collections 'staff' and 'ciarp'.")

In [ ]:
%%time
main("UNAL_13_01_2025_filtered.xlsx")

In [ ]:
# Crear STAFF y CIARP
import pandas as pd
from pymongo import MongoClient

def get_mongo_client(uri="mongodb://localhost:27017/"):
    client = MongoClient(uri)
    return client

def export_collections_to_excel(uri="mongodb://localhost:27017/", db_name="my_database",
                                staff_filename="formato_talento_humano_unal_2025_02.xlsx", ciarp_filename="formato_CIARP_UNAL_2025_02.xlsx"):
    # Conectar a MongoDB y seleccionar la base de datos
    client = get_mongo_client(uri)
    db = client[db_name]
    
    # Seleccionar las colecciones
    staff_collection = db["staff"]
    ciarp_collection = db["ciarp"]

    # Leer todos los documentos de la colección staff
    staff_docs = list(staff_collection.find({}))
    if staff_docs:
        staff_df = pd.DataFrame(staff_docs)
        if "_id" in staff_df.columns:
            staff_df.drop(columns=["_id"], inplace=True)
        if "author_full_name" in staff_df.columns:
            staff_df.drop(columns=["author_full_name"], inplace=True)
        # Guardar el DataFrame en un archivo Excel
        staff_df.to_excel(staff_filename, index=False)
        print(f"Archivo generado: {staff_filename}")
    else:
        print("La colección 'staff' está vacía.")

    # Leer todos los documentos de la colección ciarp
    ciarp_docs = list(ciarp_collection.find({}))
    if ciarp_docs:
        ciarp_df = pd.DataFrame(ciarp_docs)
        if "_id" in ciarp_df.columns:
            ciarp_df.drop(columns=["_id"], inplace=True)
        ciarp_df.to_excel(ciarp_filename, index=False)
        print(f"Archivo generado: {ciarp_filename}")
    else:
        print("La colección 'ciarp' está vacía.")

# Puedes ajustar la URI, el nombre de la base de datos y los nombres de archivos según tu entorno.
export_collections_to_excel(uri="mongodb://localhost:27017/", db_name="parser_unal",
                            staff_filename="formato_talento_humano_unal_2025_02.xlsx", ciarp_filename="formato_CIARP_UNAL_2025_02.xlsx")